In [1]:
%reload_ext autoreload
%autoreload 1
import sys
sys.path.insert(0, "../")

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from utils.loader import load

In [2]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()

In [3]:
print(train_input.shape)
print(train_target.shape)
print(train_classes.shape)
print(test_input.shape)
print(test_target.shape)
print(test_classes.shape)

torch.Size([1000, 2, 14, 14])
torch.Size([1000])
torch.Size([1000, 2])
torch.Size([1000, 2, 14, 14])
torch.Size([1000])
torch.Size([1000, 2])


In [4]:
def convert_to_one_hot_labels(target):
    tmp = torch.empty(target.shape[0],2)
    tmp[:,0] = target
    tmp[:,1] = 1 - target
    return tmp

In [5]:
train_target_one_hot = convert_to_one_hot_labels(train_target)

In [6]:
#Normalize the dataset
mean, std = train_input.mean(), train_input.std()

train_input = train_input.sub_(mean).div_(std)
test_input = test_input.sub_(mean).div_(std)

In [7]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 2)
        self.fc1 = nn.Linear(64, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 2)
         
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2))
        x = F.relu(self.fc1(x.view(-1,64)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
def train(model,train_input,train_target, criterion = nn.MSELoss(),
          optimizer =optim.SGD,batch_size=100,eta = 0.1, nb_epochs = 100) :
    
    # Define the loss and optimizing algorithm
    optimizer = optimizer(model.parameters(), lr = eta)
    # Train the model
    for e in range(nb_epochs):
        running_loss = 0.0
        for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))):
            output = model(input_)
            loss = criterion(output, target)
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}   Loss {:.6f}'.format(e,running_loss ))

In [11]:
model = LeNet()
train(model,train_input,train_target_one_hot)

Epoch: 0   Loss 2.759783
Epoch: 1   Loss 2.474296
Epoch: 2   Loss 2.451608
Epoch: 3   Loss 2.424509
Epoch: 4   Loss 2.389571
Epoch: 5   Loss 2.346220
Epoch: 6   Loss 2.298005
Epoch: 7   Loss 2.300551
Epoch: 8   Loss 2.317957
Epoch: 9   Loss 2.269802
Epoch: 10   Loss 2.239126
Epoch: 11   Loss 2.204640
Epoch: 12   Loss 2.170974
Epoch: 13   Loss 2.138062
Epoch: 14   Loss 2.111151
Epoch: 15   Loss 2.083922
Epoch: 16   Loss 2.041439
Epoch: 17   Loss 2.039976
Epoch: 18   Loss 1.964479
Epoch: 19   Loss 1.858805
Epoch: 20   Loss 2.009126
Epoch: 21   Loss 1.999865
Epoch: 22   Loss 1.826746
Epoch: 23   Loss 1.746110
Epoch: 24   Loss 1.983508
Epoch: 25   Loss 1.881182
Epoch: 26   Loss 1.751429
Epoch: 27   Loss 1.659356
Epoch: 28   Loss 1.797182
Epoch: 29   Loss 1.871837
Epoch: 30   Loss 1.742756
Epoch: 31   Loss 1.345182
Epoch: 32   Loss 1.930056
Epoch: 33   Loss 1.702348
Epoch: 34   Loss 1.256894
Epoch: 35   Loss 1.859226
Epoch: 36   Loss 1.650365
Epoch: 37   Loss 1.263670
Epoch: 38   Loss 1.509

In [9]:
def compute_nb_errors_MSE(model, test_input, test_target ) :
    # compute the output of the train model with the test set
    test_output = model(test_input)
    nb_test_errors = 0
    pred = test_output.max(1)[1]# output of the NN is 10 value for each class take the max and the [] stand for the indice of the class
    for n in range(test_input.size(0)) :
        idx = pred[n]
        if test_target[n, idx] < 0.5 : nb_test_errors = nb_test_errors + 1 # if the the predicted value is correct then in the target it is 0.9 o/w 0.0
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),nb_test_errors, test_input.size(0)))

In [10]:
def compute_nb_errors_CrossEntropy2(model, data_input, data_target) :
    nb_errors,mini_batch_size = 0, 100

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = output.max(1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors

In [11]:
def compute_nb_errors_CrossEntropy(model, test_input, test_target ) :
    # compute the output of the train model with the test set
    test_output = model(test_input)
    nb_test_errors = 0
    pred = test_output.max(1)[1]# output of the NN is 10 value for each class take the max and the [] stand for the indice of the class
    for n in range(test_input.size(0)) :
        idx = pred[n]
        if idx!= test_target[n]: nb_test_errors = nb_test_errors + 1 # if the the predicted value is correct then in the target it is 0.9 o/w 0.0
    print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors) / test_input.size(0),nb_test_errors, test_input.size(0)))

In [12]:
test_target_one_hot = convert_to_one_hot_labels(test_target)
nb_test_errors = compute_nb_errors_MSE(model, test_input, test_target_one_hot)

test error Net 21.30% 213/1000


In [13]:
model1 = LeNet()
train (model1,train_input,train_target,nn.CrossEntropyLoss())

Train Epoch: 0   Loss 6.902992
Train Epoch: 1   Loss 6.832642
Train Epoch: 2   Loss 6.777911
Train Epoch: 3   Loss 6.702267
Train Epoch: 4   Loss 6.596996
Train Epoch: 5   Loss 6.459392
Train Epoch: 6   Loss 6.359343
Train Epoch: 7   Loss 6.444841
Train Epoch: 8   Loss 6.293008
Train Epoch: 9   Loss 6.215218
Train Epoch: 10   Loss 6.156907
Train Epoch: 11   Loss 6.052761
Train Epoch: 12   Loss 5.872671
Train Epoch: 13   Loss 5.658725
Train Epoch: 14   Loss 5.822778
Train Epoch: 15   Loss 5.772002
Train Epoch: 16   Loss 5.618726
Train Epoch: 17   Loss 5.609315
Train Epoch: 18   Loss 5.508253
Train Epoch: 19   Loss 5.463863
Train Epoch: 20   Loss 5.386256
Train Epoch: 21   Loss 5.106036
Train Epoch: 22   Loss 5.180798
Train Epoch: 23   Loss 5.241637
Train Epoch: 24   Loss 4.254372
Train Epoch: 25   Loss 5.406770
Train Epoch: 26   Loss 4.993511
Train Epoch: 27   Loss 4.923427
Train Epoch: 28   Loss 4.825232
Train Epoch: 29   Loss 4.111238
Train Epoch: 30   Loss 4.789965
Train Epoch: 31   

In [36]:
nb_test_errors = compute_nb_errors_CrossEntropy(model1, test_input, test_target)

test error Net 17.30% 173/1000


In [12]:
class ResNetBlock(nn.Module):
    def __init__(self, nb_channels, kernel_size,skip_connections = True, batch_normalization = True):
        super(ResNetBlock, self).__init__()

        self.conv1 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn1 = nn.BatchNorm2d(nb_channels)

        self.conv2 = nn.Conv2d(nb_channels, nb_channels,
                               kernel_size = kernel_size,
                               padding = (kernel_size - 1) // 2)

        self.bn2 = nn.BatchNorm2d(nb_channels)
        
        self.skip_connections = skip_connections
        self.batch_normalization = batch_normalization

    def forward(self, x):
        y = self.conv1(x)
        if self.batch_normalization: y = self.bn1(y)
        y = F.relu(y)
        y = self.conv2(y)
        if self.batch_normalization: y = self.bn2(y)
        if self.skip_connections: y = y + x
        y = F.relu(y)

        return y

In [13]:
class ResNet(nn.Module):

    def __init__(self, nb_residual_blocks, nb_channels,kernel_size = 3, nb_classes = 2,skip_connections = True, batch_normalization = True):
        super(ResNet, self).__init__()

        self.conv = nn.Conv2d(2, nb_channels,
                              kernel_size = kernel_size,
                              padding = (kernel_size - 1) // 2)
        self.bn = nn.BatchNorm2d(nb_channels)

        self.resnet_blocks = nn.Sequential(
            *(ResNetBlock(nb_channels, kernel_size,batch_normalization,skip_connections)
              for _ in range(nb_residual_blocks))
        )

        self.fc = nn.Linear(nb_channels, nb_classes)

    def forward(self, x):
        x = F.relu(self.bn(self.conv(x)))
        x = self.resnet_blocks(x)
        x = F.avg_pool2d(x, 14).view(-1,32)
        x = self.fc(x)
        return x

In [34]:
model2 = ResNet(nb_residual_blocks = 30, nb_channels = 32,kernel_size = 3, nb_classes = 2)
train (model2,train_input,train_target,nn.CrossEntropyLoss())

Epoch: 0   Loss 46.887879
Epoch: 1   Loss 7.820994
Epoch: 2   Loss 7.152234
Epoch: 3   Loss 6.970767
Epoch: 4   Loss 6.840898
Epoch: 5   Loss 6.739643
Epoch: 6   Loss 7.027691
Epoch: 7   Loss 6.992197
Epoch: 8   Loss 6.541876
Epoch: 9   Loss 6.390764
Epoch: 10   Loss 5.930923
Epoch: 11   Loss 5.411363
Epoch: 12   Loss 5.280518
Epoch: 13   Loss 4.926822
Epoch: 14   Loss 4.506432
Epoch: 15   Loss 4.174598
Epoch: 16   Loss 4.007925
Epoch: 17   Loss 4.132689
Epoch: 18   Loss 3.198866
Epoch: 19   Loss 3.770926
Epoch: 20   Loss 2.889369
Epoch: 21   Loss 2.509427
Epoch: 22   Loss 2.483577
Epoch: 23   Loss 2.581824
Epoch: 24   Loss 2.554514
Epoch: 25   Loss 2.371911
Epoch: 26   Loss 1.668057
Epoch: 27   Loss 1.916489
Epoch: 28   Loss 1.030010
Epoch: 29   Loss 0.541657
Epoch: 30   Loss 1.258018
Epoch: 31   Loss 2.342561
Epoch: 32   Loss 0.388738
Epoch: 33   Loss 0.182891
Epoch: 34   Loss 0.101074
Epoch: 35   Loss 0.063305
Epoch: 36   Loss 0.041428
Epoch: 37   Loss 0.024889
Epoch: 38   Loss 0.01

In [43]:
nb_test_errors2 = compute_nb_errors_CrossEntropy2(model2, test_input, test_target)

In [44]:
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors2) / test_input.size(0),nb_test_errors2, test_input.size(0)))

test error Net 19.80% 198/1000


In [45]:
print(nb_test_errors2)

198


In [14]:
class LeNet_aux(nn.Module):
    def __init__(self):
        super(LeNet_aux, self).__init__()
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 2)
        self.conv3 = nn.Conv2d(1,32,kernel_size=3)
        self.fc1 = nn.Linear(64, 200)
        self.fc2 = nn.Linear(200, 50)
        self.fc3 = nn.Linear(50, 2)
        self.fc4 = nn.Linear(64,128)
        self.fc5 = nn.Linear(128,10)
         
    def forward(self, x):
        y = x.view(-1,1,14,14)
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 2))
        x = F.relu(self.fc1(x.view(-1,64)))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        y = F.relu(F.max_pool2d(self.conv3(y), kernel_size = 3))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size = 2))
        y = F.relu(self.fc4(y.view(-1,64)))
        y = F.relu(self.fc5(y))
        return x,y

In [15]:
def train_aux(model,train_input,train_target,train_classes, criterion = nn.MSELoss(),
              optimizer =optim.SGD,batch_size=100,eta = 0.1, nb_epochs = 100) :
    
    # Define the loss and optimizing algorithm
    optimizer = optimizer(model.parameters(), lr = eta)
    # Train the model
    for e in range(nb_epochs):
        running_loss = 0.0
        for input_, target1,target2 in (zip(train_input.split(batch_size), train_target.split(batch_size),train_classes.split(batch_size))):
            outx,outy = model(input_)
            loss1 = criterion(outx, target1)
            loss2 = criterion(outy,target2.view(200,-1).squeeze())
            loss = loss1 + loss2
            running_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('Epoch: {}   Loss {:.6f}'.format(e,running_loss ))

In [16]:
model_aux = LeNet_aux()

In [17]:
train_aux(model_aux,train_input,train_target,train_classes,nn.CrossEntropyLoss(),nb_epochs=75)

Epoch: 0   Loss 29.857390
Epoch: 1   Loss 29.540755
Epoch: 2   Loss 29.111139
Epoch: 3   Loss 28.326009
Epoch: 4   Loss 27.435583
Epoch: 5   Loss 26.628631
Epoch: 6   Loss 26.254275
Epoch: 7   Loss 25.626024
Epoch: 8   Loss 25.057392
Epoch: 9   Loss 24.442166
Epoch: 10   Loss 24.305090
Epoch: 11   Loss 23.270088
Epoch: 12   Loss 23.153136
Epoch: 13   Loss 22.386861
Epoch: 14   Loss 22.242840
Epoch: 15   Loss 22.126298
Epoch: 16   Loss 21.631952
Epoch: 17   Loss 21.191758
Epoch: 18   Loss 21.211083
Epoch: 19   Loss 21.452410
Epoch: 20   Loss 20.552109
Epoch: 21   Loss 20.368007
Epoch: 22   Loss 20.753324
Epoch: 23   Loss 20.055640
Epoch: 24   Loss 19.994173
Epoch: 25   Loss 19.836569
Epoch: 26   Loss 19.647440
Epoch: 27   Loss 19.454367
Epoch: 28   Loss 19.438775
Epoch: 29   Loss 19.521401
Epoch: 30   Loss 19.469287
Epoch: 31   Loss 19.046090
Epoch: 32   Loss 18.985659
Epoch: 33   Loss 19.123031
Epoch: 34   Loss 18.671175
Epoch: 35   Loss 19.060619
Epoch: 36   Loss 18.515720
Epoch: 37  

In [18]:
def compute_nb_errors_CrossEntropy3(model, data_input, data_target) :
    nb_errors,mini_batch_size = 0, 100

    for b in range(0, data_input.size(0), mini_batch_size):
        outx,_ = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = outx.max(1)
        for k in range(mini_batch_size):
            if data_target[b + k] != predicted_classes[k]:
                nb_errors = nb_errors + 1

    return nb_errors

In [19]:
nb_test_errors3 = compute_nb_errors_CrossEntropy3(model_aux, test_input, test_target)
print('test error Net {:0.2f}% {:d}/{:d}'.format((100 * nb_test_errors3) / test_input.size(0),nb_test_errors3, test_input.size(0)))

test error Net 18.10% 181/1000
